# 1. Setup

In [1]:
%load_ext watermark

import os
from glob import glob
import numpy as np
import cftime
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cmocean.cm as cmo

%watermark -iv -co -v

Python implementation: CPython
Python version       : 3.12.10
IPython version      : 9.2.0

conda environment: cpl_ppe_co2

cftime    : 1.6.4
cmocean   : 4.0.3
matplotlib: 3.9.1
xarray    : 2025.4.0
sys       : 3.12.10 | packaged by conda-forge | (main, Apr 10 2025, 22:21:13) [GCC 13.3.0]
numpy     : 2.0.1
cartopy   : 0.24.1



In [26]:
PPEn11_bgp = os.listdir("/glade/u/home/bbuchovecky/cesm_source/cesm_coupled_PPEn11/components/clm/src/biogeophys")
CMIP6_bgp = os.listdir("/glade/u/home/bbuchovecky/cesm_source/cesm2.1.2-rc.02-derecho/components/clm/src/biogeophys")

In [28]:
set(PPEn11_bgp) - set(CMIP6_bgp)

{'BiogeophysPreFluxCalcsMod.F90',
 'InfiltrationExcessRunoffMod.F90',
 'SaturatedExcessRunoffMod.F90',
 'SnowCoverFractionBaseMod.F90',
 'SnowCoverFractionFactoryMod.F90',
 'SnowCoverFractionNiuYang2007Mod.F90',
 'SnowCoverFractionSwensonLawrence2012Mod.F90',
 'SurfaceHumidityMod.F90',
 'SurfaceWaterMod.F90',
 'WaterBalanceType.F90',
 'WaterDiagnosticBulkType.F90',
 'WaterDiagnosticType.F90',
 'WaterFluxBulkType.F90',
 'WaterFluxType.F90',
 'WaterInfoBaseType.F90',
 'WaterInfoBulkType.F90',
 'WaterInfoIsotopeType.F90',
 'WaterInfoTracerType.F90',
 'WaterStateBulkType.F90',
 'WaterTracerContainerType.F90',
 'WaterTracerUtils.F90',
 'WaterType.F90',
 'Wateratm2lndBulkType.F90',
 'Wateratm2lndType.F90',
 'Waterlnd2atmBulkType.F90',
 'Waterlnd2atmType.F90'}

In [29]:
set(CMIP6_bgp) - set(PPEn11_bgp)

{'CanopyTemperatureMod.F90', 'WaterfluxType.F90'}

In [ ]:
# CMIP6 AMIP 2deg on Derecho

case = "f.e21.FHIST_BGC.f19_f19_mg17.CMIP6-AMIP-2deg-1month"
indir = "/glade/derecho/scratch/bbuchovecky/archive"

cam_h0 = xr.open_dataset(f"{indir}/{case}/atm/hist/{case}.cam.h0.1950-01.nc")
cam_h1 = xr.open_dataset(f"{indir}/{case}/atm/hist/{case}.cam.h1.1950-01.nc")
cam_h2 = xr.open_dataset(f"{indir}/{case}/atm/hist/{case}.cam.h2.1950-01-01-00000.nc")

clm_h0 = xr.open_dataset(f"{indir}/{case}/lnd/hist/{case}.clm2.h0.1950-02-01-00000.nc")
clm_h1 = xr.open_dataset(f"{indir}/{case}/lnd/hist/{case}.clm2.h1.1950-02-01-00000.nc")
clm_h2 = xr.open_dataset(f"{indir}/{case}/lnd/hist/{case}.clm2.h2.1950-01-01-00000.nc")

/glade/derecho/scratch/bbuchovecky/tmp/ipykernel_23601/2761534967.py:7: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  clm_h0 = xr.open_dataset(f"{indir}/{case}/lnd/hist/{case}.clm2.h0.1950-02-01-00000.nc")


In [21]:
landarea = clm_h0.area * clm_h0.landfrac
landweights = landarea / landarea.sum(dim=["lat", "lon"])

In [22]:
(clm_h0.TLAI * landweights).sum(dim=["lat", "lon"])

<xarray.DataArray (time: 1)> Size: 4B
array([1.0759795], dtype=float32)
Coordinates:
  * time     (time) object 8B 1950-02-01 00:00:00

In [ ]:
case = "f.e21.FHIST_BGC.f19_f19_mg17.CMIP6-AMIP-2deg-3year-srcmod"
indir = f"/glade/derecho/scratch/bbuchovecky/archive{case}"



In [ ]:
variables = {
    "atm": ["TREFHT", "LHFLX"],
    # "lnd": ["TLAI", "GPP", "HTOP", "TSA", "WOODC", "LEAFC", "FROOTC", "LIVESTEMC", "DEADSTEMC", "LIVECROOTC", "DEADCROOTC", "TOTLITC", "CWDC", "TOTVEGC", "TOTECOSYSC", "HR", "TOTSOMC"],
    "lnd": ["TLAI", "GPP", "HTOP", "TSA"],
}

# default simulation of historical AMIP PPE
hist_dir = "/glade/derecho/scratch/bbuchovecky/archive"
hist_case = "f.e22.FHIST_BGC.f19_f17_mg17.coupPPE-hist.000"

# 3 member ensemble of historical AMIP CMIP simulations
cmip_dir = "/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6"
cmip_case = "f.e21.FHIST_BGC.f19_f19_mg17.CMIP6-AMIP-2deg"

# default simulation of preindustrial slab ocean PPE (Claire's PPE)
piso_dir = "/glade/campaign/cgd/tss/czarakas/CoupledPPE/coupled_simulations"
piso_case = "COUP0000_PI_SOM"

start_year = 1950
end_year = 2014
times = np.array([cftime.DatetimeNoLeap(year, month, 1)
                  for year in range(start_year, end_year + 1)
                  for month in range(1, 13)])

cmip = dict()
hist = dict()
piso = dict()

for comp, vars in variables.items():
    for v in vars:
        print(v)

        hist[v] = xr.open_mfdataset(f"{hist_dir}/{hist_case}/{comp}/proc/tseries/month_1/*h0*.{v}.*.nc")[v].sel(time=slice("1950-01", "2015-01"))
        assert len(times) == hist[v].sizes["time"]
        hist[v] = hist[v].assign_coords(time=times)
        if comp == "lnd":
            hist[v] = hist[v].reindex_like(hist[variables["atm"][0]], method="nearest", tolerance=1e-3)

        if glob(f"{piso_dir}/{piso_case}/{comp}/proc/tseries/*h0*{v}.nc"):
            piso[v] = xr.open_mfdataset(f"{piso_dir}/{piso_case}/{comp}/proc/tseries/*h0*{v}.nc")[v].sel(time=slice("0100-01", "0188-12")).reindex(lat=hist[variables["atm"][0]].lat, lon=hist[variables["atm"][0]].lon, method="nearest", tolerance=1e-3)
            if comp == "lnd":
                piso[v] = piso[v].reindex(lat=hist[variables["atm"][0]].lat, lon=hist[variables["atm"][0]].lon, method="nearest", tolerance=1e-3)
        
        cmip_list = []
        for i in range(3):
            cmip_tmp = xr.open_mfdataset(f"{cmip_dir}/{cmip_case}.00{i+1}/{comp}/proc/tseries/month_1/*h0*.{v}.*.nc")[v].sel(time=slice("1950-01", "2015-01"))
            assert len(times) == cmip_tmp.sizes["time"]
            cmip_tmp = cmip_tmp.assign_coords(time=times)
            cmip_list.append(cmip_tmp.reindex_like(hist[v], method="nearest", tolerance=1e-3))
        cmip[v] = cmip_list
    
cmip_tmp = xr.open_mfdataset(f"{cmip_dir}/{cmip_case}.001/lnd/proc/tseries/month_1/*h1*.HTOP.*.nc").isel(time=0).reindex_like(hist[variables["atm"][0]], method="nearest", tolerance=1e-3)
cmip["landarea"] = cmip_tmp["area"] * cmip_tmp["landfrac"]
cmip["landweights"] = cmip["landarea"] / cmip["landarea"].sum(dim=["lat", "lon"])

hist_tmp = xr.open_mfdataset(f"{hist_dir}/{hist_case}/lnd/proc/tseries/month_1/*h1*.HTOP.*.nc").reindex(lat=hist[variables["atm"][0]].lat, lon=hist[variables["atm"][0]].lon, method="nearest", tolerance=1e-3)
hist["landarea"] = hist_tmp["area"] * hist_tmp["landfrac"]
hist["landweights"] = hist["landarea"] / hist["landarea"].sum(dim=["lat", "lon"])

piso["landarea"] = hist["landarea"]
piso["landweights"] = hist["landweights"]

area = xr.open_mfdataset(f"{cmip_dir}/{cmip_case}.001/atm/proc/tseries/month_1/*h0*.AREA.*.nc")["AREA"].isel(time=-1).reindex_like(hist[variables["atm"][0]], method="nearest", tolerance=1e-3)
areaweights = area / area.sum(dim=["lat", "lon"])

cmip["areaweights"] = areaweights
hist["areaweights"] = areaweights
piso["areaweights"] = areaweights